In [1]:
# Clonar YOLOv5 e instalar dependências
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

# Conectar Google Drive
from google.colab import drive
drive.mount('/content/drive')


Cloning into 'yolov5'...
remote: Enumerating objects: 17608, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17608 (delta 7), reused 2 (delta 2), pack-reused 17593 (from 2)
Receiving objects: 100% (17608/17608), 16.82 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (11983/11983), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import os
from pathlib import Path

# Definir caminhos
dataset_base = '/content/drive/MyDrive/FarmTechSolutions'
images_path = os.path.join(dataset_base, 'images')
labels_path = os.path.join(dataset_base, 'labels')

# Criar arquivo data.yaml para YOLO
data_yaml = f"""
train: {images_path}/train
val: {images_path}/val
test: {images_path}/test
nc: 2
names: ['ObjetoA', 'ObjetoB']
"""

with open(f'{dataset_base}/data.yaml', 'w') as f:
    f.write(data_yaml)

print("Arquivo data.yaml criado com sucesso!")


Arquivo data.yaml criado com sucesso!


In [ ]:
# Treinamento 30 épocas
!python train.py --img 640 --batch 8 --epochs 30 \
--data /content/drive/MyDrive/FarmTechSolutions/data.yaml \
--weights yolov5s.pt \
--project /content/drive/MyDrive/FarmTechSolutions/results \
--name exp30

# Treinamento 60 épocas
!python train.py --img 640 --batch 8 --epochs 60 \
--data /content/drive/MyDrive/FarmTechSolutions/data.yaml \
--weights yolov5s.pt \
--project /content/drive/MyDrive/FarmTechSolutions/results \
--name exp60


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-10-15 02:47:38.763869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760496458.789412    1375 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760496458.797268    1375 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
from IPython.display import Image, display
import glob

test_images = glob.glob(f'{images_path}/test/*/*.jpg')  # Ajuste para extensão das suas imagens

for img in test_images[:5]:  # mostrar os 5 primeiros
    !python detect.py --weights /content/drive/MyDrive/FarmTechSolutions/results/exp60/weights/best.pt \
    --img 640 --source {img} --save-txt --save-conf
    display(Image(filename=f'runs/detect/exp/{Path(img).name}'))


In [ ]:
# ========================================
# ENTREGA 2 – Comparação de Abordagens em Visão Computacional
# ========================================

# 1️⃣ Configuração inicial
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

import os
from pathlib import Path
import glob
from IPython.display import Image, display

# Caminhos
dataset_base = '/content/drive/MyDrive/FarmTechSolutions'
images_path = os.path.join(dataset_base, 'images')
labels_path = os.path.join(dataset_base, 'labels')
results_path = os.path.join(dataset_base, 'results')

# Criar data.yaml para YOLO
data_yaml = f"""
train: {images_path}/train
val: {images_path}/val
test: {images_path}/test
nc: 2
names: ['ObjetoA', 'ObjetoB']
"""
with open(f'{dataset_base}/data.yaml', 'w') as f:
    f.write(data_yaml)

print("Arquivo data.yaml criado com sucesso!")

# ========================================
# 2️⃣ Treinamento – YOLO customizado (Entrega 1)
# ========================================
!python train.py --img 640 --batch 8 --epochs 30 \
--data {dataset_base}/data.yaml \
--weights yolov5s.pt \
--project {results_path} \
--name exp30_custom

# ========================================
# 3️⃣ Treinamento – YOLO padrão
# ========================================
!python train.py --img 640 --batch 8 --epochs 30 \
--data {dataset_base}/data.yaml \
--weights yolov5s.pt \
--project {results_path} \
--name exp30_standard

# ========================================
# 4️⃣ Treinamento – CNN do zero
# ========================================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pré-processamento de imagens
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(images_path, 'train'),
    target_size=(64,64),
    batch_size=16,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(images_path, 'val'),
    target_size=(64,64),
    batch_size=16,
    class_mode='categorical'
)

# Construção do modelo CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=20)

# Salvar modelo
model.save(os.path.join(results_path, 'cnn_from_scratch.h5'))

# ========================================
# 5️⃣ Inferência e visualização de resultados
# ========================================
# YOLO customizado e YOLO padrão
test_images = glob.glob(f'{images_path}/test/*/*.jpg')

for img in test_images[:5]:
    !python detect.py --weights {results_path}/exp30_custom/weights/best.pt \
    --img 640 --source {img} --save-txt --save-conf
    display(Image(filename=f'runs/detect/exp/{Path(img).name}'))

    !python detect.py --weights {results_path}/exp30_standard/weights/best.pt \
    --img 640 --source {img} --save-txt --save-conf
    display(Image(filename=f'runs/detect/exp/{Path(img).name}'))

# CNN
import numpy as np
from tensorflow.keras.preprocessing import image

for img_path in test_images[:5]:
    img_obj = image.load_img(img_path, target_size=(64,64))
    x = image.img_to_array(img_obj)/255.0
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    classe = np.argmax(pred)
    print(f"Imagem {Path(img_path).name} - Classe CNN: {classe}")

# ========================================
# 6️⃣ Análise crítica (Markdown)
# ========================================

"""
### Comparação entre abordagens

| Critério                     | YOLO Customizado | YOLO Padrão | CNN do Zero |
|-------------------------------|----------------|------------|-------------|
| Facilidade de uso             | Alta, já adaptada para o dataset | Alta, simples de treinar | Média, precisa de pré-processamento manual |
| Precisão do modelo            | Depende do ajuste, geralmente alta | Boa, mas menos específica | Variável, depende da quantidade de dados |
| Tempo de treinamento          | Médio (~30-60 epochs) | Médio (~30 epochs) | Médio a alto (depende do tamanho da CNN) |
| Tempo de inferência           | Rápido, detecção em objetos múltiplos | Rápido | Médio, apenas classificação única |

**Observações:**
- A YOLO customizada geralmente se adapta melhor ao dataset específico.
- A YOLO padrão funciona bem “out-of-the-box”, mas pode ter limitações em casos específicos.
- A CNN do zero é interessante para classificação simples, mas não realiza detecção de múltiplos objetos na mesma imagem.
"""

